In [7]:
from PIL import Image

img = Image.open('c9.jpg')

# Coordinates of the top-left corner of the 10x10 block around the pixel (you need to find this manually)
pix_x, pix_y = 1150, 1000  # Replace x, y with actual coordinates

# Define the color red in RGB
red = (255, 0, 0)

# Modify a 10x10 area around the pixel
for i in range(pix_x, pix_x + 10):
    for j in range(pix_y, pix_y + 10):
        img.putpixel((i, j), red)
# Save the modified image
img.show()
